# Introduction

Stuart is the leading on-demand solution powering the way goods are transported in a
customised way. We connect businesses across all industries and of all sizes to high quality
independent couriers to offer customised delivery solutions.

As a data analyst at Stuart your role is to abstract the complexity of the business and the
underlying data to provide meaningful insights to decision makers. To assess your suitability
to the role, we’re providing you with a realistic data set and questions similar to what you
may encounter in the day to day work at Stuart


# Data
The data set attached consists of records of the principal business events generated from
the moment a package appears in our system until it is delivered to its final destination.

There are 22 tables in the data set recording events around 5 principle objects in Stuart’s
universe, those are: packages, deliveries, tasks, drivers and invitations.

- Packages are the physical articles that need to be delivered and can very in size,
type and urgency
- Deliveries represent the job a driver needs to perform to deliver one or more
packages
- Tasks are the different segments of a delivery. For example: a simple delivery
consists of two tasks: a pickup task and a drop off task
- Drivers are the persons performing the deliveries and can do so using different
transport types
- Invitations are the outcome of our dispatching algorithms to connect free Drivers to
Deliveries

The data is provided in an SQLite format and can be queried using any of the free sqlite
clients such as DB Browser or TablePlus

Please note that no data dictionary is provided and you’ll need to make sense of the data
based on the table names and the above information.


## Question 1
What is the success rate of packages per Zone? (% of Packages delivered on time)

### Solution 1

The query below highlights the success rate across the zones with Manchester being top and the lowest being Paris. 

In [ ]:
/**SQL QUERY**/
SELECT pd.zone_id,
       p.zone_name,
       Count(p.created_at) as count,
       Round(( Sum(CASE
                     WHEN pd.created_at <= time_window_do_end THEN 1
                     ELSE 0
                   end) * 1.0 / Count(p.created_at) ) * 100, 1) AS success_rate
FROM   package p
       INNER JOIN package_delivered pd
               ON pd.package_id = p.package_id
     where pd.zone_id is not 79 -- removed as 1 package is delivered and skews the results (shows 100% success rate)
GROUP  BY pd.zone_id
ORDER  BY success_rate DESC 

## Question 2
Is the driver invitation acceptance rate higher or lower among drivers who have been
on the platform for longer?

### Solution 2

Running the query below, the general trend is the longer users have been on the platform, the higher the acceptence rate is.

In [ ]:
/**SQL QUERY **/
SELECT Round(Avg(a.rate_accepted), 0) AS avg_rate_accepted,
       a.years_worked
FROM   (SELECT
d.driver_id,
Count(ia.delivery_invitation_id)
       AS total_invites_accept,
Count(ic.delivery_invitation_id),
(
Count(ia.delivery_invitation_id) * 1.0 / Count(ic.delivery_invitation_id) ) * 100 AS rate_accepted,
        --DATE('now'),
        --d.created_at,
        Julianday('now') - Julianday(d.created_at),
        ( CASE
            WHEN ( Julianday('now') - Julianday(d.created_at) ) < 365
          THEN
            '0-1 Year'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 365 AND
                 730 THEN
            '1-2 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 730 AND
                 1095
          THEN
            '2-3 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 1095 AND
                 1460
          THEN
            '3-4 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 1460 AND
                 1825
          THEN
            '4-5 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 1825 AND
                 2190
          THEN
            '5-6 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) BETWEEN
                 2190 AND
                 2555
          THEN
            '6-7 Years'
            WHEN ( Julianday('now') - Julianday(d.created_at) ) > 2555
          THEN
            '7+ Years'
          end )
               AS years_worked
 FROM   driver d
        INNER JOIN invitation_created ic
                ON d.driver_id = ic.driver_id
        LEFT JOIN invitation_accepted ia
               ON ic.delivery_invitation_id = ia.delivery_invitation_id
 GROUP  BY d.driver_id) a
GROUP  BY a.years_worked
ORDER  BY a.years_worked ASC 

## Question 3

What might be other relevant metrics to track? (Propose 2 metrics at least, and explain how they can be computed from the provided data).

### Solution 3

#### Metric 1: 
This allows business users to see how many people are currently only as a % on the given day. In the query below I have used as an example date to showcase  the results but effectively the metric will use Date('Now') and users would refresh to see the current status of the day. 


In [ ]:
/**SQL QUERY**/

SELECT round((sum(off_on)*1.0/count(driver_id))*100,0)
              || '%' AS percent_online_now
FROM   (
                       SELECT DISTINCT driver_id ,
                                       created_at,
                                       1 AS off_on
                       FROM            driver_online
                       UNION
                       SELECT DISTINCT driver_id,
                                       created_at,
                                       0 AS off_on
                       FROM            driver_offline ) a
WHERE  created_at BETWEEN datetime('now','start of day') AND datetime('now','LOCALTIME')

### Solution 3

#### Metric 2: 
Average Delivery Time 

This reflects the current average delivery time in minutes across all the zones


In [ ]:
select
ROUND(AVG((JULIANDAY(ts.created_at) - JULIANDAY(t.created_at))*3600),0) as Average_Delivery_Time_Minutes
from task t 
inner join task_succeeded ts 
on t.task_id = ts.task_id
where type = 'DropoffTask'

## Question 4

As a BI Analyst you are asked to create a new table in the DWH optimized for
Tableau consumption in order to build a dashboard to track driver performances. The
current event level data is too granular and unwieldy.
1.  Propose a new flat table schema to consolidate the relevant events,
2.  Write the necessary SQL to build that table,
3.  Explain the methodology you used.

### Solution 4.1

Please see attached in the assets folder the diagram required for the question

### Solution 4.2

A CTE Statement was applied below to optimise the query while joining the core base tables together. 

In [ ]:
CREATE TABLE model_driver_performance AS
             with cte_base            AS
             (
                        SELECT     distance,
                                   delivery_created_id,
                                   ad.driver_id,
                                   ad.task_id,
                                   package_id,
                                   delivery_id, (
                                   CASE
                                              WHEN type = 'PICKUP' THEN 1
                                              ELSE 0
                                   end) AS num_pickup, (
                                   CASE
                                              WHEN type = 'DROPOFF' THEN 1
                                              ELSE 0
                                   end) AS num_dropoff
                        FROM       assign_driver ad
                        INNER JOIN delivery_created dc
                        ON         ad.task_id = dc.task_id
             )
             ,
             cte_final AS
             (
                        SELECT     cb.*, (
                                   CASE
                                              WHEN status = 'delivered' THEN 1
                                              ELSE 0
                                   end) AS delivered, (
                                   CASE
                                              WHEN status = 'not_delivered' THEN 1
                                              ELSE 0
                                   end) AS not_delivered
                        FROM       cte_base cb
                        INNER JOIN package p
                        ON         cb.package_id = p.package_id
             )
  SELECT   driver_id,
           task_id,
           package_id,
           sum(distance)      AS total_distance,
           sum(delivered)     AS total_delivered,
           sum(not_delivered) AS total_not_delivered,
           (
                  SELECT sum(delivered)
                  FROM   cte_final) AS total_delivery_stuart,
           sum(num_pickup)          AS total_pickup,
           sum(num_dropoff)         AS total_dropoff
  FROM     cte_final
  GROUP BY driver_id

### Solution 4.3

I essentially wanted to create a fact table which contains the aggregated values of key measure which would highlight driver performance.  Thinking it through the flow of a driver from when they get assigned, accept the invite, and deliver (or not) the package. This drived the foreign keys used in the fact table where the corresponding primary keys would essentially be the Dimensions which would allow drilled down analysis in Tableau by the business users if required.  

This was done by creating a transaction fact table using CTE statements.Another way is to create the fact table, similarly, assigning the foreign Keys also containing a Surrogate key, the measures which are calculated are loaded via the ETL process. Or alternately to create a view or materialized view. 

## Question 5

The data provided is missing one critical component and that is weather data.
Weather plays a critical role in the availability of drivers as well as on the success of
deliveries. Using any of the free weather APIs available (e.g. DarkSky, Weatherbit,
Accuweather,...etc.) collect weather data for the locations and dates of the provided
deliveries, proposing a new table schema to store them and make them accessible to
all analysts at Stuart

### Solution

For the given question there are three key elements that need to be done, some of which are demonstrated below.
The process of extracting external data and making it available would consist of three key stages:

1. Extract the data using the AccuWeatherMap APIs (Demonstrated below in the code block)
    - Define the cities to extract the weather data for in a list
    - Run the function to extract the data
    - Store the data in a DataFrame
    - Modify the DataFrame to add an extract timestamp and city
    - Store the data in a flat file
2. Load the data into a DB
3. (Optional) Transform or refine the data

In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime

# Variables
api_key = ''
locations = {
    'Paris': [48.856613, 2.352222],
    'Barcelona': [41.3874, 2.1686],
    'Brighton': [50.8225, 0.1372],
    'Lyon': [45.7640, 4.8357],
    'Madrid': [40.4168, 3.7038],
    'Manchester': [53.4804, 2.2426],
    }

current_conditions = []


# Functions
def extract_weather(api_key, lat, lon):
    """ Extract current weather data for a single location.
            Args:
                api_key: Your personal API key
                lat: Lattitude
                lon: Longitude
            Returns:
                Flattened JSON object"""

    url = \
        'https://api.openweathermap.org/data/2.5/onecall?lat=%s&lon=%s&appid=%s&units=metric' \
        % (lat, lon, api_key)
    response = requests.get(url)
    data = json.loads(response.text)
    return data['current']


def extract_city_weather():
    """ Creates a DataFrame and appends the current weather for cities defined in a list."""
    for key in locations:
        temp = {'City': key, 'Date': datetime.timestamp(datetime.now())}
        temp.update(extract_weather(api_key, locations[key][0],
                    locations[key][1]))
        current_conditions.append(temp)


extract_city_weather()

df = pd.DataFrame(current_conditions)
print df

# The query will be run daily, initially no historical data will be present, this would be built up over time.
# Users can join through other tables through the City column via zone_name/zone_id


## Question 6

Please see the ppt deck within the assets folder. Speaker Notes have been provided at the bottom to present alongside the slides. 